<a href="https://colab.research.google.com/github/mashhadjamal/AI_REPO_074/blob/main/finetuned_txt_summarization_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate rouge_score accelerate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=41a5aaa39448e80fabc543d09b8d7e63a8d30099b830c0a05e59d256a9c6c9e5
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency re

In [2]:
import transformers

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset
ds = load_dataset("tau/multi_news")
ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


0000.parquet:   0%|          | 0.00/296M [00:00<?, ?B/s]

0001.parquet:   0%|          | 0.00/28.4M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/39.6M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/40.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

In [6]:
ds = load_dataset("tau/multi_news", split = "train[:1%]")

ds

Dataset({
    features: ['document', 'summary'],
    num_rows: 450
})

In [7]:
ds = ds.train_test_split(test_size=0.2, seed=42)

In [8]:
def clean_txt(example):
    for txt in ["document", "summary"]:  # Adjusted for the dataset fields
        example[txt] = example[txt].lower()
        example[txt] = example[txt].replace("\\", "")
        example[txt] = example[txt].replace("/", "")
        example[txt] = example[txt].replace("\n", "")
        example[txt] = example[txt].replace("``", "")
        example[txt] = example[txt].replace('"', '')
        example[txt] = example[txt].replace("--", "")

    return example


In [9]:
cleaned_ds=ds.map(clean_txt)


Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

In [10]:
EXAMPLE_TEXT_INDEX = 7
example_text = cleaned_ds["test"]["document"][EXAMPLE_TEXT_INDEX]

In [11]:
MODEL_NAME = "t5-small"

In [13]:
from transformers import pipeline

prefix = "summarize: "
summarizer = pipeline("summarization", model = MODEL_NAME, truncation=True)


summary_txt = summarizer(prefix + example_text)

summary_txt

model.safetensors:  39%|###8      | 94.4M/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

[{'summary_text': 'gao report found that nearly a third of student loan debt expected to be repaid via income-driven repayments will be forgiven by the federal government through programs such as public service loan forgiveness . the obama administration made enrolling student loan borrowers a major tool for assisting borrowers struggling with student loans debt . nearly 4.6 million federal direct borrowers had enrolled in income driven repayment plans by december 2015 up 48 percent from 2014 and 140 percent from 2013 .'}]

In [14]:
import evaluate

rouge = evaluate.load("rouge")

rouge

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [15]:
article_texts = cleaned_ds["test"]["document"]
article_summaries = cleaned_ds["test"]["summary"]

candidate_summaries = []

In [34]:
article_texts[7]

"the education department drastically underestimated the cost of the government's income-driven repayment plans for student loans in its original estimates, the government accountability office said in a highly critical report wednesday.    the gao report finds that estimates of the cost to the government of income-driven repayment plans  which eventually discharge a student's remaining debt after 20 years or more of payments  has jumped from $28 billion to $53 billion for student loans issued from 2009 to 2016. and it found that nearly a third of student loan debt expected to be repaid via income-driven repayments ($108 billion) will be forgiven by the federal government through programs such as public service loan forgiveness.    observers of federal financial aid policy said those numbers  and the apparent inability of the department to estimate the cost of student loan programs  could provide new ammunition to republican lawmakers looking to target expensive student aid programs to

In [16]:
from tqdm import tqdm



prefix = "summarize: "

for i, text in enumerate(tqdm(article_texts[:5])):

    candidate = summarizer(prefix + text)

    candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 5/5 [00:42<00:00,  8.54s/it]


In [17]:
result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_summaries[:5],
                           use_stemmer = True)
result_agg



{'rouge1': 0.24877396346215125,
 'rouge2': 0.11044815811534522,
 'rougeL': 0.14709159431096336,
 'rougeLsum': 0.1455653427847118}

In [18]:
result_unagg = rouge.compute(predictions = candidate_summaries,
                             references = article_summaries[:5],
                             use_stemmer = True,
                             use_aggregator = False)
result_unagg

{'rouge1': [0.30344827586206896,
  0.11063829787234043,
  0.2905982905982906,
  0.2695924764890282,
  0.26785714285714285],
 'rouge2': [0.0979020979020979,
  0.02575107296137339,
  0.14655172413793102,
  0.15141955835962143,
  0.125748502994012],
 'rougeL': [0.13793103448275862,
  0.059574468085106386,
  0.1623931623931624,
  0.21316614420062696,
  0.15476190476190474],
 'rougeLsum': [0.13793103448275862,
  0.059574468085106386,
  0.1623931623931624,
  0.21316614420062696,
  0.15476190476190474]}

In [19]:
import numpy as np

result_unagg_rsum = np.array(result_unagg["rouge2"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(3, 1)

In [20]:
prefix = "summarize: "


def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length = 1024, truncation = True)

    labels = tokenizer(text_target = examples["summary"], max_length = 128, truncation = True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [22]:

from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [23]:
tokenized_ds = cleaned_ds.map(preprocess_function, batched = True)

tokenized_ds

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 360
    })
    test: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 90
    })
})

In [24]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = MODEL_NAME)

In [25]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    result = rouge.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)

    prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["generated_length"] = np.mean(prediction_length)

    return {k: round(v, 4) for k, v in result.items()}


In [26]:
from huggingface_hub import notebook_login

notebook_login()

In [28]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "multinews_summary_model_trained_on_reduced_data",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 3,
    predict_with_generate = True,
    fp16 = True,
    push_to_hub = True,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)


In [29]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Generated Length
1,No log,3.160985,0.140100,0.053200,0.111500,0.111500,19.000000
2,No log,3.004454,0.139700,0.049300,0.111700,0.111600,19.000000
3,No log,2.980857,0.139900,0.049200,0.111600,0.111400,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=135, training_loss=3.4202223036024306, metrics={'train_runtime': 8337.3324, 'train_samples_per_second': 0.13, 'train_steps_per_second': 0.016, 'total_flos': 292338291179520.0, 'train_loss': 3.4202223036024306, 'epoch': 3.0})

In [30]:
trainer.push_to_hub()

events.out.tfevents.1727547421.e42c60900fcc.175.0:   0%|          | 0.00/7.91k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mashhadjamal/multinews_summary_model_trained_on_reduced_data/commit/99e7cbdeb5a60e754490f23dd4a91630774477e7', commit_message='End of training', commit_description='', oid='99e7cbdeb5a60e754490f23dd4a91630774477e7', pr_url=None, pr_revision=None, pr_num=None)